In [1]:
import torch
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [2]:
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
roberta.eval()

Using cache found in /home/henrike/.cache/torch/hub/pytorch_fairseq_main
2024-04-07 12:03:48 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


Unable to build Cython components. Please make sure Cython is installed if the torch.hub model you are loading depends on it.


2024-04-07 12:03:49 | INFO | fairseq.file_utils | loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.large.mnli.tar.gz from cache at /home/henrike/.cache/torch/pytorch_fairseq/7685ba8546f9a5ce1a00c7a6d7d44f7e748d22681172f0f391c3d48f487c801c.74e37d47306b3cc51c5f8d335022a392c29f1906c8cd9e9cd3446d7422cf55d8
/home/henrike/my-python3.8-mnli-modeltest/lib/python3.8/site-packages/hydra/experimental/initialize.py:43: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  deprecation_warning(message=message)
/home/henrike/my-python3.8-mnli-modeltest/lib/python3.8/site-packages/hydra/experimental/initialize.py:45: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  self.delegate = real_initialize(
/home/henrike/my-python3.8-mnli-modeltest/lib/python3.8/site-packages/hydra/experimental/compose.py:25: UserWarning: hydra.experimental.c

RobertaHubInterface(
  (model): RobertaModel(
    (encoder): RobertaEncoder(
      (sentence_encoder): TransformerEncoder(
        (dropout_module): FairseqDropout()
        (embed_tokens): Embedding(50265, 1024, padding_idx=1)
        (embed_positions): LearnedPositionalEmbedding(514, 1024, padding_idx=1)
        (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0-23): 24 x TransformerEncoderLayerBase(
            (self_attn): MultiheadAttention(
              (dropout_module): FairseqDropout()
              (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
       

In [3]:
locution_dataset = pd.read_csv("../../ProSeCCo/fullCorpus/ProSeCCo_final.tsv", sep="\t")
proposition_dataset = pd.read_csv("../../ProSeCCo/fullCorpus/ProSeCCo_final.tsv", sep="\t")

In [4]:
locutions1 = list(locution_dataset["locution1"])
locutions2 = list(locution_dataset["locution2"])

propositions1 = list(proposition_dataset["proposition1"])
propositions2 = list(proposition_dataset["proposition2"])

In [5]:
all_loc_preds = []

for i in range(3):
    locution_preds = []

    for loc1, loc2 in zip(locutions1, locutions2):
        tokens = roberta.encode(loc1, loc2)
        result = roberta.predict("mnli", tokens).argmax()
        locution_preds.append(int(result))

    locution_dataset["roberta_mnli_locution_pred"] = locution_preds
    locution_dataset.to_csv(f"roberta-large-mnli-results/roberta-large-mnli_locution-predictions_{i+1}.tsv", sep="\t", index=False)
    all_loc_preds.append(locution_preds)

In [6]:
all_prop_preds = []

for i in range(3):
    proposition_preds = []

    for prop1, prop2 in zip(propositions1, propositions2):
        tokens = roberta.encode(prop1, prop2)
        result = roberta.predict("mnli", tokens).argmax()
        proposition_preds.append(int(result))

    proposition_dataset["roberta_mnli_proposition_pred"] = proposition_preds
    proposition_dataset.to_csv(f"roberta-large-mnli-results/roberta-large-mnli_proposition-predictions_{i+1}.tsv", sep="\t", index=False)

    all_prop_preds.append(proposition_preds)

In [9]:
loc_accs = []
loc_f1s = []

prop_accs = []
prop_f1s = []

for i in range(1,4):
    locution_dataset = pd.read_csv(f"roberta-large-mnli-results/roberta-large-mnli_locution-predictions_{i}.tsv", sep="\t")
    proposition_dataset = pd.read_csv(f"roberta-large-mnli-results/roberta-large-mnli_proposition-predictions_{i}.tsv", sep="\t")

    loc_preds = [1 if lab==0 else 0 for lab in locution_dataset["roberta_mnli_locution_pred"]]
    loc_gold = [1 if lab=="contradiction" else 0 for lab in locution_dataset["label"]]

    prop_preds = [1 if lab==0 else 0 for lab in proposition_dataset["roberta_mnli_proposition_pred"]]
    prop_gold = [1 if lab=="contradiction" else 0 for lab in proposition_dataset["label"]]

    loc_accs.append(accuracy_score(loc_gold, loc_preds))
    loc_f1s.append(f1_score(loc_gold, loc_preds))
    
    prop_accs.append(accuracy_score(prop_gold, prop_preds))
    prop_f1s.append(f1_score(prop_gold, prop_preds))


In [10]:
print(loc_accs)

[0.5320754716981132, 0.5320754716981132, 0.5320754716981132]


In [11]:
from statistics import mean
import numpy as np

print("results with locutions:",
      "accuracy:", mean(loc_accs), "+-", np.std(np.asarray(loc_accs)), "\n",
      "f1:", mean(loc_f1s), "+-", np.std(np.asarray(loc_f1s))
)


print("results with propositions:",
      "accuracy:", mean(prop_accs), "+-", np.std(np.asarray(prop_accs)), "\n",
      "f1:", mean(prop_f1s), "+-", np.std(np.asarray(prop_f1s))
)

results with locutions: accuracy: 0.5320754716981132 +- 0.0 
 f1: 0.4483985765124555 +- 5.551115123125783e-17
results with propositions: accuracy: 0.570566037735849 +- 0.0 
 f1: 0.5004389815627743 +- 0.0
